In [1]:
import arcpy
import os
import sys
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws 
arcpy.env.overwriteOutput = True

In [2]:
floodplain = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\FloodPlain\Hydrogeo_III_merged.shp'
clarkLands = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClarkCo\TaxLots\ClarkCoTaxLots.shp'
clarkPubLands = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClarkCo\Ownership\PublicLandOwnership.shp'


## Create all county (except Clark) floodplain intersection layer with update parcel information

    This layer will be use to calculate public land are percent



In [51]:
##Correct clark county field name typo
arcpy.FeatureClassToGeodatabase_conversion (r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClarkCo\TaxLots\ClarkCoTaxLots.shp', ws)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb'>

In [80]:
### Clark public lands percentage will need to be calculate separately so first must create layer that is of all county parcels except clark
outfc = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb\allCountiesExceptClark2'

clat = 'clatsopParcels'
col = 'colParcels'
cow = 'cowlitzParcels'
mult = 'multnomahParcels'
pac = 'pacificParcels'
ska = 'skaParcels'
wah = 'wahkiakumParcels'
clark = 'ClarkCoTaxLots'

coList = [clat, col, cow, mult, pac, ska, wah, clark]
coListNoClark = [clat, col, cow, mult, pac, ska, wah]

clatOwn = 'OWNER_LINE'
colOwn = 'OWNER'
cowOwn = 'DEED_HOLDE'
multOwn = 'OWNER1'
pacOwn = 'CURRENT_O'
skaOwn = 'OwnerLast'
wahOwn = 'Property_O'
clarkOwn = 'LegalShort'

ownList = [clatOwn, colOwn, cowOwn, multOwn, pacOwn, skaOwn, wahOwn, clarkOwn]
ownListNoClark = [clatOwn, colOwn, cowOwn, multOwn, pacOwn, skaOwn, wahOwn]

publicLand = 'PublicLand'
AREA = 'Shape_Area'
LENGTH = 'Shape_Length'




In [5]:
for co in coList:
    spatial_ref = arcpy.Describe(co).spatialReference
    print(f'{co}:{spatial_ref.name}')
   

clatsopParcels:NAD_1983_HARN_StatePlane_Oregon_North_FIPS_3601_Feet_Intl
colParcels:NAD_1983_HARN_StatePlane_Oregon_North_FIPS_3601_Feet_Intl
cowlitzParcels:NAD_1983_HARN_StatePlane_Washington_South_FIPS_4602_Feet
multnomahParcels:WGS_1984_Web_Mercator_Auxiliary_Sphere
pacificParcels:NAD_1983_StatePlane_Washington_South_FIPS_4602_Feet
skaParcels:NAD_1983_HARN_StatePlane_Washington_South_FIPS_4602_Feet
wahkiakumParcels:NAD_1983_HARN_StatePlane_Washington_South_FIPS_4602_Feet
clarkParcels:NAD_1983_StatePlane_Washington_South_FIPS_4602_Feet


In [6]:
# Reproject multnomah county to appropriate projection
arcpy.Project_management(mult, 'multnomahParcel', arcpy.Describe(clat).spatialReference)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\multnomahParcel'>

In [7]:
publicLandFldMap = arcpy.FieldMap()
shapeAreaFldMap = arcpy.FieldMap()
shapeLengthFldMap = arcpy.FieldMap()
landOwnerFldMap = arcpy.FieldMap()

fms = arcpy.FieldMappings()

In [76]:
for co, own in zip(coListNoClark, ownListNoClark):
    publicLandFldMap.addInputField(co, publicLand)
    shapeAreaFldMap.addInputField(co, AREA)
    shapeLengthFldMap.addInputField(co, LENGTH)
    landOwnerFldMap.addInputField(co, own)
    

In [9]:
landOwner_name = landOwnerFldMap.outputField
landOwner_name.name = 'Land Owner'
landOwner_name.length = 100
landOwnerFldMap.outputField = landOwner_name

In [10]:
area_name = shapeAreaFldMap.outputField
area_name.name = 'AREA'
shapeAreaFldMap.outputField = area_name

length_name = shapeLengthFldMap.outputField
length_name.name = 'LENGTH'
shapeLengthFldMap.outputField = length_name

In [11]:
fms.addFieldMap(publicLandFldMap)
fms.addFieldMap(shapeAreaFldMap)
fms.addFieldMap(shapeLengthFldMap)
fms.addFieldMap(landOwnerFldMap)

In [20]:
arcpy.Merge_management(coListNoClark, outfc, fms)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\allCountiesExceptClark2'>

In [21]:
## Create floodplain intersection
newParcelsExceptClark_FP = arcpy.SelectLayerByLocation_management(outfc, 'INTERSECT', floodplain)
arcpy.CopyFeatures_management(newParcelsExceptClark_FP, 'newParcelsExceptClark_FP')

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\newParcelsExceptClark_FP'>

## Create all county (including Clark) floodplain intersection with updated parcel layers
    
    This layer will be used to compare total area of new floodplain intersection with old floodplain intersection
    


In [105]:
## MAke new all county layer that includes all parcels including calrk county

shapeAreaFldMap = arcpy.FieldMap()
shapeLengthFldMap = arcpy.FieldMap()
landOwnerFldMap = arcpy.FieldMap()

fms = arcpy.FieldMappings()

In [106]:
for co, own in zip(coList, ownList):
    shapeAreaFldMap.addInputField(co, AREA)
    shapeLengthFldMap.addInputField(co, LENGTH)
    #print(co, own)
    landOwnerFldMap.addInputField(co, own)

In [107]:
landOwner_name = landOwnerFldMap.outputField
landOwner_name.name = 'Land Owner'
landOwner_name.length = 500
landOwnerFldMap.outputField = landOwner_name

In [108]:
fms.addFieldMap(shapeAreaFldMap)
fms.addFieldMap(shapeLengthFldMap)
fms.addFieldMap(landOwnerFldMap)

In [109]:
arcpy.Merge_management(coList, 'county_merge_allParcels', fms)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\county_merge_allParcels'>

In [110]:
# Create later of new parcels that intersect with floodplain
newFP_allParcels = arcpy.SelectLayerByLocation_management('county_merge_allParcels', "INTERSECT", floodplain)
arcpy.CopyFeatures_management(newFP_allParcels, 'newFP_allParcels')

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\newFP_allParcels'>

In [111]:
fields = arcpy.ListFields('newFP_allParcels')

for field in fields:
    print(field.name)

OBJECTID
Shape
Land_Owner
Shape_Length
Shape_Area


## Creat all county floodplain intersection with outdated parcel layers

In [112]:
# Repeat process for old parcels
oldFP_allParcels = arcpy.SelectLayerByLocation_management('old_parcels_merge', 'INTERSECT', floodplain)
arcpy.CopyFeatures_management(oldFP_allParcels, 'oldFP_allParcels')

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\oldFP_allParcels'>

## After inspecting oldFP_allParcels and newFP_allParcels, it looks like the same features have multiple entries in the attribute table so need write function to identify and eliminate double entries.

In [113]:
arcpy.DeleteIdentical_management('newFP_allParcels', ['Shape_Area', 'Shape_Length', 'Land_Owner'])

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\newFP_allParcels'>

In [114]:
arcpy.DeleteIdentical_management('oldFp_allParcels', ['Shape_Area', 'Shape_Length', 'Land_Owner'])

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\oldFp_allParcels'>

In [68]:
arcpy.DeleteIdentical_management('newParcelsExceptClark_FP', ['AREA', 'LENGTH', 'Land_Owner'])

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\newParcelsExceptClark_FP'>

## Calculate areas of both new and outdated all county floodplain intersections

In [27]:
def calcArea(fc, areaField):
    with arcpy.da.SearchCursor(fc, areaField) as cursor:
        area = 0
        for row in cursor:
            area = area + row[0]
    return area

In [131]:
newArea = calcArea('newFP_allParcels', 'Shape_Area')
oldArea = calcArea('oldFP_allParcels', 'Shape_Area')

print(f'New Area: {newArea} sq ft \nOld Area: {oldArea} sq ft')

percentChange = (newArea/oldArea * 100) - 100
print(f'The updated tax parcel has a {round(percentChange, 2)} % larger area than the old tax parcel layer')

New Area: 20175681785.65799 sq ft 
Old Area: 19186135269.893784 sq ft
The updated tax parcel has a 5.16 % larger area than the old tax parcel layer


## Calculate Percentage of Public Lands

In [135]:
newPubLands = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLotDatabaseCreate\TaxLotDatabaseCreate.gdb\PublicLandsFloodplain'
oldPubLands = 'pub_pvt_exportpub_wm_Project'

newPubArea = calcArea(newPubLands, 'Shape_Area')
oldPubArea = calcArea(oldPubLands, 'Shape_Area')

print(f'Updated public lands have an area of {newPubArea} sq ft \nOutdated public lands have an area of {oldPubArea} sq ft')

percentNewPubLands = newPubArea/newArea  * 100
percentOldPubLands = oldPubArea/oldArea * 100

print(f'The updated layer has {percentNewPubLands} % public lands\nThe outdated layer has {percentOldPubLands} % public lands')


Updated public lands have an area of 4003732811.8900023 sq ft 
Outdated public lands have an area of 3565880272.7875705 sq ft
The updated layer has 19.84434952149216 % public lands
The outdated layer has 18.58571422866504 % public lands


In [ ]:
## Prepare clark county layer

arcpy.AddField_management(clarkPubLands,'PublicLand', 'TEXT', field_length = 1)

with arcpy.da.UpdateCursor(clarkPubLands, 'PublicLand') as cursor:
    for row in cursor:
        row[0] = 'T'
        cursor.updateRow(row)

In [ ]:
#Prepare for Merge
publicLandFldMap = arcpy.FieldMap()
fms = arcpy.FieldMappings()


In [ ]:
publicLandFldMap.addInputField(clarkPubLands, 'PublicLand')
pubLand_name = publicLandFldMap.outputField
pubLand_name.name = 'PublicLand'
pubLand_name.length = 1
publicLandFldMap.outputField = pubLand_name

fms.addFieldMap(publicLandFldMap)


In [ ]:
arcpy.SpatialJoin_analysis(clarkLands, clarkPubLands, 'clarkParcelsPubLands', field_mapping = fms,
                          match_option = 'HAVE_THEIR_CENTER_IN')

In [ ]:
join_matches = arcpy.SelectLayerByAttribute_management('clarkParcelsPubLands', where_clause = "PublicLand = 'T'")
arcpy.CopyFeatures_management(join_matches, 'join_matches')


In [ ]:
omittedParcels = arcpy.SelectLayerByLocation_management(clarkPubLands, 'INTERSECT', 'join_matches', invert_spatial_relationship = 'INVERT')
arcpy.CopyFeatures_management(omittedParcels, 'omittedParcels')

In [ ]:
# Create later of new parcels that intersect with floodplain
newFP_allParcels = arcpy.SelectLayerByLocation_management('county_merge', "INTERSECT", floodplain)
arcpy.CopyFeatures_management(newFP_allParcels, 'newFP_allParcels')

In [ ]:
def percPubLandArea(layer, ):
    